In [15]:
# !pip install -r ./requirements.txt

In [2]:
# !pip uninstall virny -y

In [4]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@development

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@development

In [5]:
# !pip install lightgbm==3.3.5

In [6]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [7]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [8]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /home/dh3553/projects/fairness-variance


## Import dependencies

In [9]:
import os
import copy

from virny.utils.custom_initializers import create_config_obj
from virny.datasets import ACSIncomeDataset

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS
from configs.models_config_for_tuning import get_folktables_employment_models_params_for_tuning

from source.preprocessing import get_simple_preprocessor
from source.experiment_interface import run_exp_iter_with_preprocessing_intervention

## Define Input Variables

In [10]:
# ROOT_DIR = os.path.join(os.getcwd(), "..", "..")
ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'mult_repair_levels_income_GA'
DB_COLLECTION_NAME = 'exp_mult_repair_levels'
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME)
FAIR_INTERVENTION_PARAMS_LST = [0.4, 0.3, 0.2, 0.0]

config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', EXPERIMENT_NAME, 'folk_GA_2018_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

## Define a db writer and custom fields to insert into your database

In [11]:
import os
from dotenv import load_dotenv

load_dotenv('./configs/secrets.env')
os.getenv("DB_NAME")

'fairness_variance'

In [12]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [13]:
import uuid

custom_table_fields_dct = {
#     'session_uuid': str(uuid.uuid4()),
    'session_uuid': '78c3da49-e095-43f3-8cc1-fa34bab0357c',
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  78c3da49-e095-43f3-8cc1-fa34bab0357c


## Initialize custom objects

In [14]:
data_loader = ACSIncomeDataset(state=['GA'], year=2018, with_nulls=False,
                               subsample_size=20_000, subsample_seed=42)
data_loader.X_data.head()

,SCHL,COW,MAR,OCCP,POBP,RELP,SEX,RAC1P,AGEP,WKHP
0,23,7,3,230,36,0,1,1,55,55.0
1,16,1,5,4110,13,2,2,1,20,35.0
2,16,4,3,4130,51,0,2,1,59,30.0
3,18,4,1,4020,13,0,1,2,43,40.0
4,14,1,1,8300,20,1,2,2,33,20.0


In [15]:
data_loader.X_data.shape

(20000, 10)

In [16]:
# Check if RAC1P 4 is in the X_data
data_loader.X_data.RAC1P.unique()

array(['1', '2', '6', '9', '8', '7', '3', '5'], dtype=object)

## Run experiment iterations

### Experiment iteration 1

In [12]:
# tuned_params_filenames = ['tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv']
# tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
#                          for tuned_params_filename in tuned_params_filenames]

In [13]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [14]:
run_exp_iter_with_preprocessing_intervention(data_loader=exp_iter_data_loader,
                                             experiment_seed=experiment_seed,
                                             test_set_fraction=TEST_SET_FRACTION,
                                             db_writer_func=db_writer_func,
                                             fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                             column_transformer=preprocessor,
                                             models_params_for_tuning=models_params_for_tuning,
                                             metrics_computation_config=metrics_computation_config,
                                             custom_table_fields_dct=custom_table_fields_dct,
                                             with_tuning=True,
#                                              with_tuning=False,
#                                              tuned_params_df_paths=tuned_params_df_paths,
                                             save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                             verbose=True)

2023-07-18 05:38:17 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'experiment_iteration': 'Exp_iter_1',
 'fair_intervention_params_lst': '[1.0]',
 'model_init_seed': 100,
 'session_uuid': '78c3da49-e095-43f3-8cc1-fa34bab0357c'}




2023-07-18 05:38:17 experiment_interface.py INFO    : The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([ 2917,  2234, 14396,  1781, 11102,   732,  7692, 10589, 16098,
             4920,  6601,  7611,  3825, 18755,  6862,  3847,  7256, 13711,
            12389,  9772],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([ 2917,  2234, 14396,  1781, 11102,   732,  7692, 10589, 16098,
             4920,  6601,  7611,  3825, 18755,  6862,  3847,  7256, 13711,
            12389,  9772],
           dtype='int64')


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

...

2023-07-18 06:17:24 experiment_interface.py INFO    : Models are tuned and saved to a file


2023/07/18, 06:17:24: Tuning for RandomForestClassifier is finished [F1 score = 0.7851744243391806, Accuracy = 0.8099999999999999]



Multiple runs progress:   0%|          | 0/1 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 2

In [17]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_GA_2018_Income_alpha_0.4_20230718__141431.csv',
    'tuning_results_Folktables_GA_2018_Income_alpha_0.3_20230718__141438.csv',
    'tuning_results_Folktables_GA_2018_Income_alpha_0.2_20230717__222835.csv',
    'tuning_results_Folktables_GA_2018_Income_alpha_0.0_20230717__215443.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_preprocessing_intervention(data_loader=exp_iter_data_loader,
                                             experiment_seed=experiment_seed,
                                             test_set_fraction=TEST_SET_FRACTION,
                                             db_writer_func=db_writer_func,
                                             fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                             column_transformer=preprocessor,
                                             models_params_for_tuning=models_params_for_tuning,
                                             metrics_computation_config=metrics_computation_config,
                                             custom_table_fields_dct=custom_table_fields_dct,
                                             # with_tuning=True,
                                             with_tuning=False,
                                             tuned_params_df_paths=tuned_params_df_paths,
                                             save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                             verbose=True)

2023-07-18 11:29:32 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:


{'dataset_split_seed': 200,
 'experiment_iteration': 'Exp_iter_2',
 'fair_intervention_params_lst': '[0.5, 0.6, 0.7, 0.8, 0.9, 1.0]',
 'model_init_seed': 200,
 'session_uuid': '78c3da49-e095-43f3-8cc1-fa34bab0357c'}




2023-07-18 11:29:32 experiment_interface.py INFO    : The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([ 4298,  9201,   679, 18780,  6955,  8571,  9405, 13699,  3930,
            13573,  3774,  9085, 12646, 16782, 13102, 18789,  8042, 19647,
            13861,  8780],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([ 4298,  9201,   679, 18780,  6955,  8571,  9405, 13699,  3930,
            13573,  3774,  9085, 12646, 16782, 13102, 18789,  8042, 19647,
            13861,  8780],
           dtype='int64')


Multiple alphas:   0%|          | 0/6 [00:00<?, ?it/s]

...

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

intervention_param:  0.7
Path for tuned params:  /home/dh3553/projects/fairness-variance/results/mult_repair_levels_income_GA/tuning_results_Folktables_GA_2018_Income_alpha_0.7_20230718__101840.csv
RandomForestClassifier:  {'bootstrap': False, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 10, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 200, 'n_jobs': None, 'oob_score': False, 'random_state': 201, 'verbose': 0, 'warm_start': False}


Multiple runs progress:   0%|          | 0/1 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 3

In [47]:
# Configs for an experiment iteration
exp_iter_num = 3
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_GA_2018_Income_alpha_0.4_20230718__141431.csv',
    'tuning_results_Folktables_GA_2018_Income_alpha_0.3_20230718__141438.csv',
    'tuning_results_Folktables_GA_2018_Income_alpha_0.2_20230717__222835.csv',
    'tuning_results_Folktables_GA_2018_Income_alpha_0.0_20230717__215443.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [5]:
run_exp_iter_with_preprocessing_intervention(data_loader=exp_iter_data_loader,
                                             experiment_seed=experiment_seed,
                                             test_set_fraction=TEST_SET_FRACTION,
                                             db_writer_func=db_writer_func,
                                             fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                             column_transformer=preprocessor,
                                             models_params_for_tuning=models_params_for_tuning,
                                             metrics_computation_config=metrics_computation_config,
                                             custom_table_fields_dct=custom_table_fields_dct,
                                             # with_tuning=True,
                                             with_tuning=False,
                                             tuned_params_df_paths=tuned_params_df_paths,
                                             save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                             verbose=True)

### Experiment iteration 4

In [64]:
# Configs for an experiment iteration
exp_iter_num = 4
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_GA_2018_Income_alpha_0.4_20230718__141431.csv',
    'tuning_results_Folktables_GA_2018_Income_alpha_0.3_20230718__141438.csv',
    'tuning_results_Folktables_GA_2018_Income_alpha_0.2_20230717__222835.csv',
    'tuning_results_Folktables_GA_2018_Income_alpha_0.0_20230717__215443.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [6]:
run_exp_iter_with_preprocessing_intervention(data_loader=exp_iter_data_loader,
                                             experiment_seed=experiment_seed,
                                             test_set_fraction=TEST_SET_FRACTION,
                                             db_writer_func=db_writer_func,
                                             fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                             column_transformer=preprocessor,
                                             models_params_for_tuning=models_params_for_tuning,
                                             metrics_computation_config=metrics_computation_config,
                                             custom_table_fields_dct=custom_table_fields_dct,
                                             # with_tuning=True,
                                             with_tuning=False,
                                             tuned_params_df_paths=tuned_params_df_paths,
                                             save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                             verbose=True)

### Experiment iteration 5

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 5
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_GA_2018_Income_alpha_0.4_20230718__141431.csv',
    'tuning_results_Folktables_GA_2018_Income_alpha_0.3_20230718__141438.csv',
    'tuning_results_Folktables_GA_2018_Income_alpha_0.2_20230717__222835.csv',
    'tuning_results_Folktables_GA_2018_Income_alpha_0.0_20230717__215443.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_preprocessing_intervention(data_loader=exp_iter_data_loader,
                                             experiment_seed=experiment_seed,
                                             test_set_fraction=TEST_SET_FRACTION,
                                             db_writer_func=db_writer_func,
                                             fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                             column_transformer=preprocessor,
                                             models_params_for_tuning=models_params_for_tuning,
                                             metrics_computation_config=metrics_computation_config,
                                             custom_table_fields_dct=custom_table_fields_dct,
                                             # with_tuning=True,
                                             with_tuning=False,
                                             tuned_params_df_paths=tuned_params_df_paths,
                                             save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                             verbose=True)

### Experiment iteration 6

In [17]:
# Configs for an experiment iteration
exp_iter_num = 6
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_GA_2018_Income_alpha_0.4_20230718__141431.csv',
    'tuning_results_Folktables_GA_2018_Income_alpha_0.3_20230718__141438.csv',
    'tuning_results_Folktables_GA_2018_Income_alpha_0.2_20230717__222835.csv',
    'tuning_results_Folktables_GA_2018_Income_alpha_0.0_20230717__215443.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [18]:
run_exp_iter_with_preprocessing_intervention(data_loader=exp_iter_data_loader,
                                             experiment_seed=experiment_seed,
                                             test_set_fraction=TEST_SET_FRACTION,
                                             db_writer_func=db_writer_func,
                                             fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                             column_transformer=preprocessor,
                                             models_params_for_tuning=models_params_for_tuning,
                                             metrics_computation_config=metrics_computation_config,
                                             custom_table_fields_dct=custom_table_fields_dct,
                                             # with_tuning=True,
                                             with_tuning=False,
                                             tuned_params_df_paths=tuned_params_df_paths,
                                             save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                             verbose=True)

2023-07-18 14:25:06 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:


{'dataset_split_seed': 600,
 'experiment_iteration': 'Exp_iter_6',
 'fair_intervention_params_lst': '[0.4, 0.3, 0.2, 0.0]',
 'model_init_seed': 600,
 'session_uuid': '78c3da49-e095-43f3-8cc1-fa34bab0357c'}




2023-07-18 14:25:06 experiment_interface.py INFO    : The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([12903, 10978, 15717,   120, 11708, 17066, 10746, 14041, 10990,
            15799, 14577,  6418,  6874, 17667,  7093,  3725, 10105,  5805,
            19952,  7399],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([12903, 10978, 15717,   120, 11708, 17066, 10746, 14041, 10990,
            15799, 14577,  6418,  6874, 17667,  7093,  3725, 10105,  5805,
            19952,  7399],
           dtype='int64')


Multiple alphas:   0%|          | 0/4 [00:00<?, ?it/s]

...

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

intervention_param:  0.0
Path for tuned params:  /home/dh3553/projects/fairness-variance/results/mult_repair_levels_income_GA/tuning_results_Folktables_GA_2018_Income_alpha_0.0_20230717__215443.csv
RandomForestClassifier:  {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 100, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 10, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 1000, 'n_jobs': None, 'oob_score': False, 'random_state': 601, 'verbose': 0, 'warm_start': False}


Multiple runs progress:   0%|          | 0/1 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]